In [1]:
# import necessary packages
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import json

In [6]:
test_dict = {
    "name": "Buzz",
    "rollno": 1,
    "gpa": 4.0,
}

with open("test.json", "w") as testf:
    json.dump(test_dict, testf)

In [7]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 701C-0AC8

 Directory of C:\Users\Potato\Desktop\PYTHON STUFF\fuck it up\SCHOOL\FALL 2022\SD\GT-Senior-Design\LinUCB_modeling

09/15/2022  09:53 PM    <DIR>          .
09/15/2022  09:53 PM    <DIR>          ..
09/15/2022  09:51 PM    <DIR>          .ipynb_checkpoints
09/15/2022  09:30 PM             3,396 disjoint_LinUCB.py
09/15/2022  09:51 PM            49,403 disjoint_LinUCB_starter.ipynb
09/15/2022  09:53 PM                41 test.json
               3 File(s)         52,840 bytes
               3 Dir(s)  350,816,907,264 bytes free


In [9]:
with open('test.json', 'r') as testf:
    # Reading from json file
    test_val = json.load(testf)

print(test_val)

{'name': 'Buzz', 'rollno': 1, 'gpa': 4.0}


In [10]:
updated_test_dict = {
    "name": "Buzz",
    "rollno": 1,
    "gpa": 3.9,
}

with open("test.json", "w") as testf:
    json.dump(updated_test_dict, testf)

In [11]:
with open('test.json', 'r') as testf:
    # Reading from json file
    test_val = json.load(testf)

print(test_val)

{'name': 'Buzz', 'rollno': 1, 'gpa': 3.9}


In [13]:
# global constants
ARMS = ["Academic Services", "Career Exploration and Coaching", "Wellness Enhancement Opportunities",
        "Physical Wellbeing Resources", "Mindfulness Resources", "Social Wellbeing and Engagement",
        "Spiritual Engagement and Volunteering", "Disability Services", "Academic Support - Dean of Students",
        "One-on-One Consultation", "Other"]
CONTEXT_DIM = 25
alpha = 0.05

A = {arm: np.identity(CONTEXT_DIM).tolist() for arm in ARMS}
b = {arm: np.zeros(CONTEXT_DIM).tolist() for arm in ARMS}

In [14]:
A

{'Academic Services': [[1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,

In [15]:
b

{'Academic Services': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Career Exploration and Coaching': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Wellness Enhancement Opportunities': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Physical Wellbeing Resources': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Mindfulness Resources': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0

In [16]:
with open("A.json", "w") as Afile:
    json.dump(A, Afile)
    
with open("b.json", "w") as bfile:
    json.dump(b, bfile)

In [17]:
%ls

 Volume in drive C is Windows
 Volume Serial Number is 701C-0AC8

 Directory of C:\Users\Potato\Desktop\PYTHON STUFF\fuck it up\SCHOOL\FALL 2022\SD\GT-Senior-Design\LinUCB_modeling

09/15/2022  10:00 PM    <DIR>          .
09/15/2022  10:00 PM    <DIR>          ..
09/15/2022  09:51 PM    <DIR>          .ipynb_checkpoints
09/15/2022  10:00 PM            35,272 A.json
09/15/2022  10:00 PM             1,722 b.json
09/15/2022  09:30 PM             3,396 disjoint_LinUCB.py
09/15/2022  09:59 PM            58,904 disjoint_LinUCB_starter.ipynb
09/15/2022  09:55 PM                41 test.json
               5 File(s)         99,335 bytes
               3 Dir(s)  350,815,698,944 bytes free


In [18]:
arm = 'Academic Services'

theta_hat = np.linalg.solve(A[arm], b[arm])
theta_hat

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# reading JSON from file
f = open("input.json")
data = json.load(f)
f.close()

# reading JSON from a JSON string
js = "<JSON STRING OBJECT PLACEHOLDER>"
data = json.loads(js)

In [ ]:
# Global constants.
NUM_ARMS = 8       # The number of arms
# Num_Clusters = 4   # The number of clusters
CONTEXT_DIM = 25   # The dimension of context vectors
T = 20000          # The number of rounds

In [ ]:
# context is a randomly selected CONTEXT_DIM dimensional np array from the input data

In [ ]:
def train_disjoint_LinUCB(alpha, input_A=None, input_b=None):
    '''
    Input:
        alpha: exloration parameter
        input_A: input A from previous training sessions
        input_b: input b from previous training sessions
    
    Output:
        A: return this parameter so it can be saved and used in later runs if needed
        b: return this parameter so it can be saved and used in later runs if needed
        average_reward: numpy array of shape=(T,), the average reward of the algorithm after each round
        num_pulls: dict, the number of pulls of each arm after T rounds
        
    '''
    # Initialization
    average_reward = np.zeros(T)
    num_pulls = {arm: 0.0 for arm in range(1, NUM_ARMS+1)}
    
    A = {arm: np.identity(CONTEXT_DIM) for arm in range(1, NUM_ARMS+1)}
    if(input_A is not None):
        A = input_A
        
    b = {arm: np.zeros(CONTEXT_DIM) for arm in range(1, NUM_ARMS+1)}
    if(input_b is not None):
        b = input_b
    
    # Interact with the contextual bandit simulator
    for t in range(1, T+1):
        # A context is revealed to the algorithm
        # TODO: REPLACE THIS LINE WITH HOW WE ARE ACCESSING INPUT ENTRIES
        context = env.generate_context()
        
        # Choose arm = argmax UCB[i], with ties broken uniformly random
        UCB = []
        for arm in range(1, NUM_ARMS+1):
            # theta_hat = A[arm]^{-1} b[arm]
            theta_hat = np.linalg.solve(A[arm], b[arm])
            # ucb = theta_hat^T context + alpha * sqrt(context^T A[arm]^{-1} context)
            ucb = np.dot(theta_hat, context) + alpha * np.sqrt(np.dot(np.dot(context, np.linalg.inv(A[arm])), context))
            UCB.append(ucb)
            
        max_ucb = np.max(UCB)
        candidates = [arm for arm in range(1, NUM_ARMS+1) if UCB[arm-1] == max_ucb]
        arm = np.random.choice(candidates)
        
        # A reward is revealed to the algorithm
        # TODO: REPLACE THIS LINE WITH HOW WE ARE GENERATING REWARDS
        r = env.generate_reward(arm, context)
        
        # TO DO:
        # Update A and b based on the context and the reward
        # x x^T results in an d by d matrix so we need to reshape context right????
        A[arm] = A[arm] + ( context.reshape((CONTEXT_DIM,1)) @ context.reshape((1,CONTEXT_DIM)) )
        b[arm] = b[arm] + (r * context)
        
        # TO DO:
        # Update average_reward and num_pulls based on the arm pulled and the reward
        average_reward[t-1] = r if (t==1) else (average_reward[t-1-1]*(t-1) + r) / t
        num_pulls[arm] = num_pulls[arm] + 1
    
    print("------------------------------------------------------------------------------------------------------------------------")
    
    return A, b, average_reward, num_pulls

In [ ]:
def get_rec(context, input_A, input_b):
    # Choose arm = argmax UCB[i], with ties broken uniformly random
    UCB = []
    for arm in range(1, NUM_ARMS+1):
        # theta_hat = A[arm]^{-1} b[arm]
        theta_hat = np.linalg.solve(input_A[arm], input_b[arm])
        # ucb = theta_hat^T context + alpha * sqrt(context^T A[arm]^{-1} context)
        ucb = np.dot(theta_hat, context) + alpha * np.sqrt(np.dot(np.dot(context, np.linalg.inv(input_A[arm])), context))
        UCB.append(ucb)
            
    max_ucb = np.max(UCB)
    candidates = [arm for arm in range(1, NUM_ARMS+1) if UCB[arm-1] == max_ucb]
    arm = np.random.choice(candidates)
    
    return arm